# Chapter 7: Error Handling

Learn how to handle errors gracefully in Rust.

## Unrecoverable Errors with panic!

Use `panic!` for errors that should never happen.

In [ ]:
fn main() {
    // panic!("crash and burn");
    
    // Out-of-bounds access causes panic
    let v = vec![1, 2, 3];
    // v[99];  // This will panic!
}

## Recoverable Errors with Result

`Result<T, E>` is used for operations that might fail.

In [ ]:
use std::fs::File;

fn main() {
    let f = File::open("hello.txt");
    
    let f = match f {
        Ok(file) => file,
        Err(error) => {
            println!("Problem opening file: {:?}", error);
            return;
        }
    };
}

## Matching on Different Errors

In [ ]:
use std::fs::File;
use std::io::ErrorKind;

fn main() {
    let f = File::open("hello.txt");
    
    let f = match f {
        Ok(file) => file,
        Err(error) => match error.kind() {
            ErrorKind::NotFound => match File::create("hello.txt") {
                Ok(fc) => fc,
                Err(e) => panic!("Problem creating file: {:?}", e),
            },
            other_error => panic!("Problem opening file: {:?}", other_error),
        },
    };
}

## Shortcuts: unwrap and expect

`unwrap()` returns the value or panics. `expect()` lets you choose the panic message.

In [ ]:
use std::fs::File;

fn main() {
    // unwrap - panics with default message
    // let f = File::open("hello.txt").unwrap();
    
    // expect - panics with custom message
    let f = File::open("hello.txt").expect("Failed to open hello.txt");
}

## Propagating Errors

Return errors to the calling code instead of handling them directly.

In [ ]:
use std::fs::File;
use std::io::{self, Read};

fn read_username_from_file() -> Result<String, io::Error> {
    let f = File::open("hello.txt");
    
    let mut f = match f {
        Ok(file) => file,
        Err(e) => return Err(e),
    };
    
    let mut s = String::new();
    
    match f.read_to_string(&mut s) {
        Ok(_) => Ok(s),
        Err(e) => Err(e),
    }
}

fn main() {
    match read_username_from_file() {
        Ok(username) => println!("Username: {}", username),
        Err(e) => println!("Error: {:?}", e),
    }
}

## The ? Operator

The `?` operator is a shortcut for propagating errors.

In [ ]:
use std::fs::File;
use std::io::{self, Read};

fn read_username_from_file() -> Result<String, io::Error> {
    let mut f = File::open("hello.txt")?;  // Returns early if error
    let mut s = String::new();
    f.read_to_string(&mut s)?;  // Returns early if error
    Ok(s)
}

// Even more concise
fn read_username_shorter() -> Result<String, io::Error> {
    let mut s = String::new();
    File::open("hello.txt")?.read_to_string(&mut s)?;
    Ok(s)
}

// Most concise
fn read_username_shortest() -> Result<String, io::Error> {
    std::fs::read_to_string("hello.txt")
}

## Where The ? Operator Can Be Used

The `?` operator can only be used in functions that return `Result` or `Option`.

In [ ]:
use std::error::Error;
use std::fs::File;

// main can return Result
fn main() -> Result<(), Box<dyn Error>> {
    let f = File::open("hello.txt")?;
    Ok(())
}

## Using ? with Option

In [ ]:
fn last_char_of_first_line(text: &str) -> Option<char> {
    text.lines().next()?.chars().last()
}

fn main() {
    let text = "Hello\nWorld";
    match last_char_of_first_line(text) {
        Some(c) => println!("Last char: {}", c),
        None => println!("No last char"),
    }
}

## Custom Error Types

In [ ]:
use std::fmt;

#[derive(Debug)]
enum MathError {
    DivisionByZero,
    NegativeSquareRoot,
}

impl fmt::Display for MathError {
    fn fmt(&self, f: &mut fmt::Formatter) -> fmt::Result {
        match self {
            MathError::DivisionByZero => write!(f, "Cannot divide by zero"),
            MathError::NegativeSquareRoot => write!(f, "Cannot take square root of negative number"),
        }
    }
}

fn divide(a: f64, b: f64) -> Result<f64, MathError> {
    if b == 0.0 {
        Err(MathError::DivisionByZero)
    } else {
        Ok(a / b)
    }
}

fn main() {
    match divide(10.0, 2.0) {
        Ok(result) => println!("Result: {}", result),
        Err(e) => println!("Error: {}", e),
    }
}

## To panic! or Not to panic!

### Use `panic!` when:
- Code is in a bad state that should never happen
- No reasonable way to recover
- Writing examples or prototype code
- Writing tests

### Use `Result` when:
- Errors are expected and recoverable
- Writing library code
- Caller should decide how to handle errors

## Exercises

1. Write a function that parses a string to an integer and returns Result
2. Create a custom error type for a calculator
3. Chain multiple operations using the ? operator
4. Write a function that reads and parses a config file

In [ ]:
// Exercise 1: Parse integer
fn parse_integer(s: &str) -> Result<i32, std::num::ParseIntError> {
    // Your code here
    s.parse()
}

// Exercise 2: Calculator errors
#[derive(Debug)]
enum CalcError {
    // Define error variants
}

fn calculate(op: &str, a: f64, b: f64) -> Result<f64, CalcError> {
    // Your code here
    Ok(0.0)
}

## Key Takeaways

- Use `panic!` for unrecoverable errors
- Use `Result<T, E>` for recoverable errors
- The `?` operator propagates errors concisely
- `unwrap()` and `expect()` are shortcuts but can panic
- Custom error types provide better error handling
- Choose between `panic!` and `Result` based on context